### Arbitrary Shaped Room Mode Simulator (3D)

#### A Very Quick Review of the Theory
The room modes can be found by solving the [_Helmholtz equation_](https://en.wikipedia.org/wiki/Helmholtz_equation), where $\hat{p}$ is the pressure amplitude, $\nabla^2$ is the [_Laplacian_](https://en.wikipedia.org/wiki/Laplace_operator), $\displaystyle k = \frac{2 \pi f}{c}$ is the [acoustic wavenumber](http://www.tonmeister.ca/main/textbook/intro_to_sound_recordingch4.html#x19-1780003.1.15), $f$ is frequency in Hz, and $c$ is speed of sound in m/s:
<br><br>
\\[ \nabla^2 \hat{p} + k^2 \hat{p} = 0 \\] <br>
In the Cartesian coordinate system: <br><br>
\\[ \nabla^2 \equiv \frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2} \\]

Rearrange the equation into the more recognizable form of the classic eigenvalue problem:<br><br>
\\[ (-\nabla^2) \hat{p} = k^2 \hat{p} \\] <br>
After discretization, ($-\nabla^2$) becomes the negative of the Laplacian matrix. $k^2$ is the eigenvalue and $\hat{p}$ is the corresponding eigenvector (discretized eigenfunction) of the Laplacian matrix.

This simulator assumes perfect reflection of the room walls, which is reasonable since we mostly concern with the bass frequencies. This simply means that the boundary conditions are naturally applied by the FEM formulations and we don't need to specify any boundary conditions.

----
#### Start
Clear all variable definitions.

Obtain the speed of sound at sea level using standard atmospheric data.

In [ ]:
(* Clear all variable definitions *)
ClearAll["Global`*"];

(* Speed of sound at sea level standard atmosphere, in SI unit *)
c = QuantityMagnitude[StandardAtmosphereData[Quantity[0, "Meter"], "SoundSpeed"]]

#### Construct our example  room using basic 3D solid primatives
The interior volume of our example model &mdash; a house with a loft &mdash; is build using the 3D solid primatives cuboid, prism and hexahedron.

To define a cuboid: `Cuboid[p1, p2]` where `p1` is `{x1, y1, z1}` and `p2` is `{x2, y2, z2}`.

To define a prism: `Prism[p1, p2, p3, p4, p5, p6]` where `p1` is `{x1, y1, z1}`, etc.

To define a hexahedron: `Hexahedron[p1, p2, p3, p4, p5, p6, p7, p8]` where `p1` is `{x1, y1, z1}`, etc.

In [ ]:
Import["3D_primatives.png"]
Import["Loft.png"]

In [ ]:
cb1 = Cuboid[{0, 0, 0}, {5, 1, 3}];
cb2 = Cuboid[{0, 1, 0}, {4, 6, 3}];
cb3 = Cuboid[{0, 0, 3}, {10, 6, 5}];
ps1 = Prism[{{5, 0, 0}, {8, 0, 3}, {5, 0, 3},
             {5, 1, 0}, {8, 1, 3}, {5, 1, 3}}];
hx1 = Hexahedron[{{0, 2, 8}, {0, 6, 5}, {0, 0, 5}, {0, 0, 6},
                  {10, 2, 8}, {10, 6, 5}, {10, 0, 5}, {10, 0, 6}}];

#### Display the solids

In [ ]:
Graphics3D[{Opacity[0.1], {{Red, cb1}, {Green, cb2}, {Blue, cb3}, {Cyan, ps1}, {Magenta, hx1}}},
           ImageSize -> Medium, Axes -> True, AxesLabel -> {x, y, z}, ViewVector -> {15, -15, 15}]

#### Merge the solids together and generate the mesh
Display the merged solid, the FEM mesh and report the number of elements in the mesh.

In [ ]:
model = RegionUnion[cb1, cb2, cb3, ps1, hx1];

mesh = DiscretizeRegion[model, MeshQualityGoal -> "Maximal", AccuracyGoal -> 4,
                        MaxCellMeasure -> {"Volume" -> 0.01}, PerformanceGoal -> "Quality"];

Grid[{{Graphics3D[{Opacity[0.1], model}, ImageSize -> Medium, Axes -> True,
                  AxesLabel -> {x, y, z}, ViewVector -> {15, -15, 15}],
       Graphics3D[{Opacity[0.25], mesh}, ImageSize -> Medium, ViewVector -> {15, -15, 15}]}}]
MeshCellCount[mesh, 3]

#### Setup and solve the eigenvalue problem
We'll solve for the first `nmodes` modes, and report the compute time (in seconds).

In [ ]:
nmodes = 20;
AbsoluteTiming[{lambda, eigfuns} = NDEigensystem[{-Laplacian[{u[x, y, z]}, {x, y, z}]}, u[x, y, z],
                                                 Element[{x, y, z}, mesh], nmodes];
              ]

Convert eigenvalues into mode frequencies.

In [ ]:
Grid[Transpose[{Range[nmodes], freqs = c * Sqrt[lambda] / (2 * Pi)}], Alignment -> Right]

#### Visualize the room modes
Plot one of the mode shapes (mode number 12) using 3D density plot, and report the compute time.

This actually takes significantly more computation time than solving the eigenvalue problem &#128556;

In [ ]:
mode = 12;

In [ ]:
AbsoluteTiming[denplot = DensityPlot3D[eigfuns[[mode]], Element[{x, y, z}, model],
                                       ColorFunction -> ColorData["RedGreenSplit"],
                                       ViewVector -> {15, -15, 15}];
              ]

If we are using Mathematica, the 3D density plot generated is a 3D object and we can spin it around.<br>
However, Wolfram on Jupyter notebook only supports static pictures, but we can generate multiple views by specifying a series of view point locations.

In [ ]:
r = 30;
theta = Pi/3;

Table[Show[denplot, AxesLabel -> {x, y, z},  ViewVector -> FromSphericalCoordinates[{r, theta, phi}]],
      {phi, -3*Pi/4, Pi, Pi/4}]

Other methods to present the results &mdash; slice density plots.

In [ ]:
slice1 = SliceDensityPlot3D[eigfuns[[mode]], {"XStackedPlanes"}, Element[{x, y, z}, model],
                            ColorFunction -> ColorData["RedGreenSplit"],
                            ViewVector -> {15, -15, 15}];
slice2 = SliceDensityPlot3D[eigfuns[[mode]], {"YStackedPlanes"}, Element[{x, y, z}, model],
                            ColorFunction -> ColorData["RedGreenSplit"],
                            ViewVector -> {15, -15, 15}];
slice3 = SliceDensityPlot3D[eigfuns[[mode]], {"ZStackedPlanes"}, Element[{x, y, z}, model],
                            ColorFunction -> ColorData["RedGreenSplit"],
                            ViewVector -> {15, -15, 15}];

In [ ]:
Show[slice1, slice2, slice3]

A series of slices. Unfortunately, Wolfram doesn't offer slice plots with transparency.

In [ ]:
Table[SliceDensityPlot3D[eigfuns[[mode]],
                         {{"XStackedPlanes", {0}}, {"YStackedPlanes", {ycoord}}, {"ZStackedPlanes", {0}}},
                         Element[{x, y, z}, model], ColorFunction -> ColorData["RedGreenSplit"],
                         ViewVector -> {15, -15, 15}],
      {ycoord, 0, 6}]